In [12]:
pip install wikimapper

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install datapackage

Note: you may need to restart the kernel to use updated packages.


# Data Extraction Notebook

In [16]:
import pandas as pd
from pathlib import Path
import requests
import tarfile
from wikimapper import WikiMapper
from datapackage import Package
import numpy as np
import src.utils
from src.utils import *
import importlib
importlib.reload(src.utils)

<module 'src.utils' from '/Users/ghaliabennani/Desktop/MA3/ADA/ada-2024-project-ada212/src/utils.py'>

# CMU Dataset
Here we fetch from the CMU website our movie dataset and we store it in the data directory.
DELETE THE data DIRECTORY BEFORE RUNNING THE FOLLOWING CELL.

In [17]:
MOVIE_CMU_URL = "http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz"
response = requests.get(MOVIE_CMU_URL, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path='.')

Path("MovieSummaries").rename("data")
data_path = Path("data")

for file_name in ["character.metadata.tsv", "name.clusters.txt", "plot_summaries.txt", "README.txt", "tvtropes.clusters.txt"]:
    file_path = data_path / file_name
    if file_path.exists():
        file_path.unlink()

cmu_cols = ["movie_wikipedia_id", "movie_freebase_id", "movie_title", "movie_release", "movie_revenue", "movie_runtime", "movie_languages", "movie_countries", "movie_genres"]
cmu_df = (pd.read_csv(
    data_path / "movie.metadata.tsv",
    sep="\t",
    header=None,
    names=cmu_cols,
    usecols=["movie_wikipedia_id", "movie_title", "movie_release", "movie_revenue", "movie_runtime",  "movie_languages", "movie_countries", "movie_genres"])
.assign(
    movie_release=lambda df: df.movie_release.astype(str).str.slice(0, 4).replace("nan", pd.NA).astype("Int32"),
)
)

 # Wikipedia to Wikidata Ids
The CMU dataset uses wikipedia ids. To link movies with books we need the movies' wikidata ids. This is what we do in the following cells

In [18]:
!wikimapper download enwiki-latest --dir data
!wikimapper create enwiki-latest --dumpdir data --target data/index_enwiki-latest.db

2024-12-14 12:04:16,637 - wikimapper.download - INFO - Downloading [https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-page.sql.gz] to [data/enwiki-latest-page.sql.gz]
2024-12-14 12:14:31,130 - wikimapper.download - INFO - Downloading [https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-page_props.sql.gz] to [data/enwiki-latest-page_props.sql.gz]
2024-12-14 12:16:00,082 - wikimapper.download - INFO - Downloading [https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-redirect.sql.gz] to [data/enwiki-latest-redirect.sql.gz]
2024-12-14 12:16:35,422 - wikimapper.processor - INFO - Creating index for [enwiki-latest] in [data/index_enwiki-latest.db]
2024-12-14 12:16:35,423 - wikimapper.processor - INFO - Parsing pages dump
2024-12-14 12:18:50,547 - wikimapper.processor - INFO - Creating database index on 'wikipedia_title'
2024-12-14 12:19:08,029 - wikimapper.processor - INFO - Parsing page properties dump
2024-12-14 12:19:54,698 - wikimapper.processor - INFO - Parsing redirects du

In [19]:
#we put in our dataset the wikidata id corresponding to the wikipedia id we already have.
mapper = WikiMapper(data_path / "index_enwiki-latest.db")
cmu_df = (cmu_df.assign(
    movie_wikidata_id = lambda x: x.movie_wikipedia_id.apply(
        lambda wikipedia_id: mapper.wikipedia_id_to_id(wikipedia_id)
    )
)
          .drop(columns=["movie_wikipedia_id"])
          )

# Books-Movies Pairs
Now that we have the wikidata ids for the movies in our dataset we can identify the book adaptations in the wikidata database thanks to the SPARQL query in the following cell. We also extract the features of the corresponding books.

In [20]:
WIKI_DATA_SERVICE_URL = 'https://query.wikidata.org/sparql'
query = '''
SELECT DISTINCT ?movie ?book ?bookLabel ?authorLabel ?instanceOfLabel ?countryLabel ?pubDateLabel ?genreLabel ?awardLabel ?seriesLabel ?goodreadsLabel
WHERE 
{
  VALUES ?bookType { wd:Q47461344 wd:Q7725634 wd:Q571 wd:Q14406742 wd:Q21198342 wd:Q277759 }
  VALUES ?movieType { wd:Q11424 wd:Q506240 }

  ?book wdt:P31 ?bookType.
  OPTIONAL {?book wdt:P50 ?author}
  OPTIONAL {?book wdt:P31 ?instanceOf}
  OPTIONAL {?book wdt:P495 ?country}
  OPTIONAL {?book wdt:P577 ?pubDate}
  OPTIONAL {?book wdt:P136 ?genre}
  OPTIONAL {?book wdt:P166 ?award}
  OPTIONAL {?book wdt:P179 ?series}
  OPTIONAL {?book wdt:P8383 ?goodreads}

  ?movie wdt:P31 ?movieType;          
         wdt:P144 ?book.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''
query_result = requests.get(WIKI_DATA_SERVICE_URL, params = {'format': 'json', 'query': query})
wikidata_df =pd.DataFrame(query_result.json()['results']['bindings'])
for column in wikidata_df.columns:
    wikidata_df[column] = wikidata_df[column].apply(lambda x: x['value'] if isinstance(x, dict) and 'value' in x else x)

def get_list(series: pd.Series) -> list:
    return list(set(series.dropna().tolist()))

def mode(x: pd.Series) -> pd.Series:
    modes = x.mode()
    if len(modes) > 0:
        return modes.iloc[0]
    return None

categories = {
    'fiction': {'novel', 'short novel', 'novella', 'serialized fiction', 'short story', 'war fiction', 'magic realist fiction', 'metafiction', 'science fiction', 'suspense in literature', 'horror novel', 'horror fiction', 'crime fiction', 'psychological thriller', 'speculative/fantastic fiction', 'adventure fiction', 'detective fiction', 'noir fiction', 'political novel', 'vampire fiction', 'dystopian fiction', 'social science fiction', 'techno-thriller', 'thriller', 'fantasy', 'Gothic novel', 'picaresque novel', 'mystery fiction', 'post-apocalyptic fiction', 'philosophical fiction', 'romantic fiction', 'Bildungsroman', 'roman à clef', 'comedy', 'black comedy'},
    'non_fiction': {'nonfiction', 'memoir', 'autobiography', 'biographical novel', 'biography', 'essay'},
    'children': {'children\'s literature', 'children\'s fiction', 'young adult fiction', 'children\'s novel'},
    'historical': {'historical fiction', 'historical novel'},
    'drama': {'play', 'drama', 'tragedy'},
    'anime': {'adventure anime and manga', 'drama anime and manga'},
    'fantasy': {'magic realist fiction', 'fantasy', 'vampire fiction', 'fairy tale'},
    'science_fiction': {'science fiction', 'dystopian fiction', 'social science fiction', 'techno-thriller', 'post-apocalyptic fiction'},
    'horror': {'horror novel', 'horror fiction'},
    'thriller': {'psychological thriller', 'thriller'},
    'detective': {'detective fiction', 'noir fiction', 'mystery fiction', 'cloak and dagger novel'},
    'satire': {'satire', 'satirical fiction', 'metafiction'},
    'comedy': {'comedy', 'black comedy'},
}

wikidata_df = (wikidata_df
               .assign(
    movie_wikidata_id = lambda x: x.movie.str.split('/').str[-1],
    book_wikidata_id = lambda x: x.book.str.split('/').str[-1],
    book_release = lambda x: pd.to_datetime(x.pubDateLabel, errors='coerce').dt.year.astype('Int64')
)
               .groupby(['movie_wikidata_id', 'book_wikidata_id'])
               .agg(
    book_title = pd.NamedAgg(column='bookLabel', aggfunc=mode),
    book_author = ('authorLabel', 'first'),
    book_release = ('book_release', 'first'),
    book_country = ('countryLabel', 'first'),
    book_goodreads_id = ('goodreadsLabel', 'first'),
    series = ('seriesLabel', 'first'),
    instance_of = pd.NamedAgg(column='instanceOfLabel', aggfunc=get_list),
    genre = pd.NamedAgg(column='genreLabel', aggfunc=get_list),
    award = pd.NamedAgg(column='awardLabel', aggfunc=get_list)
)
               .assign(
    book_part_of_series = lambda x: x.series.notnull().astype(int),
    literary_work = lambda x: x.instance_of.apply(lambda y: 'literary work' in y).astype(int),
    written_work = lambda x: x.instance_of.apply(lambda y: 'written work' in y).astype(int),
    comic_book_seris = lambda x: x.instance_of.apply(lambda y: 'comic book series' in y).astype(int),
    book_series = lambda x: x.instance_of.apply(lambda y: 'book series' in y).astype(int),
    manga_series = lambda x: x.instance_of.apply(lambda y: 'manga series' in y).astype(int),
    book_fiction = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['fiction'])) > 0).astype(int),
    book_non_fiction = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['non_fiction'])) > 0).astype(int),
    book_children = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['children'])) > 0).astype(int),
    book_historical = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['historical'])) > 0).astype(int),
    book_drama = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['drama'])) > 0).astype(int),
    book_anime = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['anime'])) > 0).astype(int),
    book_fantasy = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['fantasy'])) > 0).astype(int),
    book_science_fiction = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['science_fiction'])) > 0).astype(int),
    book_horror = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['horror'])) > 0).astype(int),
    book_thriller = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['thriller'])) > 0).astype(int),
    book_detective = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['detective'])) > 0).astype(int),
    book_satire = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['satire'])) > 0).astype(int),
    book_comedy = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['comedy'])) > 0).astype(int),
    book_won_price = lambda x: x.award.apply(lambda y: len(y) > 0).astype(int),
)
               .drop(['instance_of', 'genre', 'award', 'series'], axis=1)
               .reset_index()
               )

/var/folders/j1/j4l0ld1s6dz8kzf6ptph9n5w0000gn/T/ipykernel_7317/1133718669.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  book_release = lambda x: pd.to_datetime(x.pubDateLabel, errors='coerce').dt.year.astype('Int64')


# Book Dataset
We create a book dataset still using the Wikidata Query Service.

In [21]:
import time
WIKI_DATA_SERVICE_URL = 'https://query.wikidata.org/sparql'
query = '''
SELECT DISTINCT ?book
WHERE 
{
  VALUES ?bookType { wd:Q47461344 wd:Q7725634 wd:Q571 wd:Q14406742 wd:Q21198342 wd:Q277759 }
  ?book wdt:P31 ?bookType.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''
query_result = requests.get(WIKI_DATA_SERVICE_URL, params = {'format': 'json', 'query': query})
book_df = pd.DataFrame(query_result.json()['results']['bindings'])
for column in book_df.columns:
    book_df[column] = book_df[column].apply(lambda x: x['value'] if isinstance(x, dict) and 'value' in x else x)
book_ids = book_df.book.str.split('/').str[-1].tolist()

def get_book_wikidata(ids, retries=5, backoff_factor=1):
    book_ids = " ".join([f"wd:{id}" for id in ids])
    query = (
            'SELECT DISTINCT ?movie ?book ?bookLabel ?authorLabel ?instanceOfLabel ?countryLabel '
            '?pubDateLabel ?genreLabel ?awardLabel ?seriesLabel '
            'WHERE { '
            'VALUES ?book {' + book_ids + '} '
                                          'OPTIONAL { ?book wdt:P31 ?instanceOf. } '
                                          'OPTIONAL { ?book wdt:P136 ?genre. } '
                                          'OPTIONAL { ?book wdt:P495 ?country. } '
                                          'OPTIONAL { ?book wdt:P577 ?pubDate. } '
                                          'OPTIONAL { ?book wdt:P166 ?award. } '
                                          'OPTIONAL { ?book wdt:P179 ?series. } '
                                          'OPTIONAL { ?book wdt:P50 ?author. } '
                                          'OPTIONAL { ?movie wdt:P144 ?book. } '
                                          'SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } '
                                          '}'
    )

    for attempt in range(retries):
        query_result = requests.get(WIKI_DATA_SERVICE_URL, params={'format': 'json', 'query': query})

        if query_result.status_code == 200:
            try:
                json_data = query_result.json()
                df = pd.DataFrame(json_data['results']['bindings'])
                for column in df.columns:
                    df[column] = df[column].apply(lambda x: x.get('value') if isinstance(x, dict) else x)
                return df
            except ValueError as e:
                print("Error decoding JSON:", e)
                return pd.DataFrame()  # Return an empty DataFrame if JSON error

        elif query_result.status_code == 429:
            # Exponential backoff
            wait_time = backoff_factor * (2 ** attempt)
            time.sleep(wait_time)
        else:
            print(f"Request failed with status code {query_result.status_code}")
            return pd.DataFrame()  # Return empty DataFrame if other errors

    print("Max retries reached.")
    return pd.DataFrame()  # Return an empty DataFrame in case of JSON error

book_df_list = []
for i in range(0, len(book_ids), 100):
    book_df_list.append(get_book_wikidata(book_ids[i:i+100]))
book_df = pd.concat(book_df_list)

In [22]:
book_df = (book_df
           .assign(
    book_wikidata_id = lambda x: x.book.str.split('/').str[-1],
    book_release = lambda x: pd.to_datetime(x.pubDateLabel, errors='coerce').dt.year.astype('Int64')
)
           .groupby(['book_wikidata_id'])
           .agg(
    book_title = pd.NamedAgg(column='bookLabel', aggfunc=mode),
    book_author = ('authorLabel', 'first'),
    book_release = ('book_release', 'first'),
    book_country = ('countryLabel', 'first'),
    series = ('seriesLabel', 'first'),
    instance_of = pd.NamedAgg(column='instanceOfLabel', aggfunc=get_list),
    genre = pd.NamedAgg(column='genreLabel', aggfunc=get_list),
    award = pd.NamedAgg(column='awardLabel', aggfunc=get_list)
)
           .assign(
    book_part_of_series = lambda x: x.series.notnull().astype(int),
    literary_work = lambda x: x.instance_of.apply(lambda y: 'literary work' in y).astype(int),
    written_work = lambda x: x.instance_of.apply(lambda y: 'written work' in y).astype(int),
    comic_book_seris = lambda x: x.instance_of.apply(lambda y: 'comic book series' in y).astype(int),
    book_series = lambda x: x.instance_of.apply(lambda y: 'book series' in y).astype(int),
    manga_series = lambda x: x.instance_of.apply(lambda y: 'manga series' in y).astype(int),
    book_fiction = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['fiction'])) > 0).astype(int),
    book_non_fiction = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['non_fiction'])) > 0).astype(int),
    book_children = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['children'])) > 0).astype(int),
    book_historical = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['historical'])) > 0).astype(int),
    book_drama = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['drama'])) > 0).astype(int),
    book_anime = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['anime'])) > 0).astype(int),
    book_fantasy = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['fantasy'])) > 0).astype(int),
    book_science_fiction = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['science_fiction'])) > 0).astype(int),
    book_horror = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['horror'])) > 0).astype(int),
    book_thriller = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['thriller'])) > 0).astype(int),
    book_detective = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['detective'])) > 0).astype(int),
    book_satire = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['satire'])) > 0).astype(int),
    book_comedy = lambda x: x.genre.apply(lambda y: len(set(y).intersection(categories['comedy'])) > 0).astype(int),
    book_won_price = lambda x: x.award.apply(lambda y: len(y) > 0).astype(int),
)
           .drop(['instance_of', 'genre', 'award', 'series'], axis=1)
           .reset_index()
           )

# Goodreads dataset
We add some elements to our book dataset, we use a Kaggle dataset curated from Goodreads. We also take care of cleaning the format of the author name and the title.

In [23]:
! mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: /Users/ghaliabennani/.kaggle: File exists
mv: kaggle.json: No such file or directory


In [24]:
!pip install kaggle

In [25]:
!kaggle datasets download -d bahramjannesarr/goodreads-book-datasets-10m
!unzip goodreads-book-datasets-10m.zip
!rm goodreads-book-datasets-10m.zip
!rm user_rating_*.csv
!mkdir data/goodreads

Dataset URL: https://www.kaggle.com/datasets/bahramjannesarr/goodreads-book-datasets-10m
License(s): CC0-1.0
100%|███████████████████████████████████████▉| 460M/460M [00:22<00:00, 30.6MB/s]
100%|████████████████████████████████████████| 460M/460M [00:22<00:00, 21.1MB/s]
Archive:  goodreads-book-datasets-10m.zip
  inflating: book1-100k.csv          
  inflating: book1000k-1100k.csv     
  inflating: book100k-200k.csv       
  inflating: book1100k-1200k.csv     
  inflating: book1200k-1300k.csv     
  inflating: book1300k-1400k.csv     
  inflating: book1400k-1500k.csv     
  inflating: book1500k-1600k.csv     
  inflating: book1600k-1700k.csv     
  inflating: book1700k-1800k.csv     
  inflating: book1800k-1900k.csv     
  inflating: book1900k-2000k.csv     
  inflating: book2000k-3000k.csv     
  inflating: book200k-300k.csv       
  inflating: book3000k-4000k.csv     
  inflating: book300k-400k.csv       
  inflating: book4000k-5000k.csv     
  inflating: book400k-500k.csv       
  i

In [26]:

!mv book*.csv data/goodreads/

In [27]:
def clean_title(title_series: pd.Series) -> pd.Series:
    return (title_series
            .str.split('(').str[0]
            .str.split(':').str[0]
            .str.lower()
            .str.replace('and', '&')
            .str.replace('.', '')
            .str.replace("'", '')
            .str.replace('-', ' ')
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
            )

def clean_author(author_series: pd.Series) -> pd.Series:
    initial_letter = (author_series
                      .str.strip()
                      .str[0]
                      .str.lower())
    last_name = (author_series
                 .str.split(r"(\s|-|')", regex=True)
                 .str[-1]
                 .str.replace('.', '')
                 .str.replace("'", '')
                 .str.replace(r'\s+', ' ', regex=True)
                 .str.strip()
                 .str.lower()
                 )
    return initial_letter + " " + last_name

In [28]:
wikidata_df = wikidata_df.assign(
    join_title = lambda x: clean_title(x.book_title),
    join_author = lambda x: clean_author(x.book_author),
)

In [29]:
book_df = book_df.assign(
    join_title = lambda x: clean_title(x.book_title),
    join_author = lambda x: clean_author(x.book_author),
)   


In [30]:
df_list = []
for file in data_path.glob('goodreads/book*.csv'):
    try:
        df = (pd.read_csv(
            file,
            usecols=['Name', 'Authors', 'Publisher', 'pagesNumber', 'Rating', 'RatingDistTotal']
        )
              .rename(columns={'pagesNumber': 'book_pages', 'RatingDistTotal': 'book_ratings_count', 'Rating': 'book_rating',
                               'Publisher': 'book_publisher', 'Authors': 'book_author',
                               'Name': 'book_title'})
              )
        df_list.append(df)
    except:
        df = (pd.read_csv(
            file,
            usecols=['Name', 'Authors', 'Publisher', 'PagesNumber', 'Rating', 'RatingDistTotal']
        )
              .rename(columns={'PagesNumber': 'book_pages', 'RatingDistTotal': 'book_ratings_count', 'Rating': 'book_rating',
                               'Publisher': 'book_publisher', 'Authors': 'book_author',
                               'Name': 'book_title'})
              )
        df_list.append(df)

In [31]:
goodreads_df = (pd.concat(df_list, ignore_index=True)
                .assign(
    book_rating = lambda x: x.book_rating.replace(0, np.nan),
    book_pages = lambda x: x.book_pages.replace(0, np.nan).astype('Int64'),
    book_ratings_count = lambda x: x.book_ratings_count.str.split(':').str[-1].astype('Int64'),
    join_title = lambda x: clean_title(x.book_title),
    join_author = lambda x: clean_author(x.book_author),
)
                .merge(
    wikidata_df.loc[:, ['join_title', 'join_author']].drop_duplicates(),
    on=['join_title', 'join_author'],
    how='inner'
)
                .drop_duplicates(subset=['join_title', 'join_author'])
                .drop(['book_title', 'book_author'], axis=1)
                )

In [32]:
goodreads_book_df = (pd.concat(df_list, ignore_index=True)
                     .assign(
    book_rating = lambda x: x.book_rating.replace(0, np.nan),
    book_pages = lambda x: x.book_pages.replace(0, np.nan).astype('Int64'),
    book_ratings_count = lambda x: x.book_ratings_count.str.split(':').str[-1].astype('Int64'),
    join_title = lambda x: clean_title(x.book_title),
    join_author = lambda x: clean_author(x.book_author),
)
                     .merge(
    book_df.loc[:, ['join_title', 'join_author']].drop_duplicates(),
    on=['join_title', 'join_author'],
    how='inner'
)
                     .drop_duplicates(subset=['join_title', 'join_author'])
                     .drop(['book_title', 'book_author'], axis=1)
                     )

In [35]:
(book_df
 .merge(goodreads_book_df, on=['join_title', 'join_author'], how='inner')
 .drop(columns=['join_title', 'join_author'])
 .to_csv("data/book.csv", index=False)
 )

# More Revenues infomations with TMDB
We have seen that there are a lot of missing values when it comes to movie revenues so we fetch some data from The Movie DataBase.

In [36]:
!kaggle datasets download -d rounakbanik/the-movies-dataset
!unzip the-movies-dataset.zip
!rm the-movies-dataset.zip
!mv movies_metadata.csv data/
!rm credits.csv
!rm keywords.csv
!rm links.csv
!rm links_small.csv
!rm ratings.csv
!rm ratings_small.csv

Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
100%|███████████████████████████████████████▊| 227M/228M [00:09<00:00, 33.0MB/s]
100%|████████████████████████████████████████| 228M/228M [00:09<00:00, 25.1MB/s]
Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [37]:
def replace_jpg(x):
    return np.nan if isinstance(x, str) and x.endswith('.jpg') else x


tmdb_df = (pd.read_csv("data/movies_metadata.csv")
           .assign(
    movie_budget = lambda df: df.budget.apply(replace_jpg).astype("Int64").replace(0, pd.NA),
    movie_revenue_tmdb = lambda df: df.revenue.replace(0.0, pd.NA).astype("Int64")
)
           .loc[:, ['imdb_id', 'movie_budget', 'movie_revenue_tmdb']]
           .drop_duplicates(subset=['imdb_id'])
           )

/var/folders/j1/j4l0ld1s6dz8kzf6ptph9n5w0000gn/T/ipykernel_7317/1954682421.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tmdb_df = (pd.read_csv("data/movies_metadata.csv")


# Making Revenues and Budgets comparable
A way to normalize the revenues and budget data and to make the values comparable across time we we adjust for inflation using the US Consumer Price.

In [38]:
import pandas as pd
cpi_df = (pd.read_csv("data_utils/flat-ui__data-Thu Nov 14 2024.csv")
          .rename(columns={'Date': 'date', 'Index': 'cpi', 'Inflation': '_'})
          .assign(
    year = lambda df: df.date.astype(str).str.slice(0, 4).astype("Int32"),
    inflation_adjustment = lambda df: (df.cpi.iloc[0] / df.cpi).astype(float)
)
         .drop(columns=['date', 'cpi', '_'])
         .drop_duplicates(subset=['year'])
         .reset_index(drop=True)
         .assign(inflation_adjustment = lambda x: x.inflation_adjustment.astype(float))
          )
cpi_df.head()

,year,inflation_adjustment
0,2014,1.000000
1,2013,1.003720
2,2012,1.018793
3,2011,1.036531
4,2010,1.067237


In [39]:
cpi_df.to_csv("data/inflation_adjustment.csv", index=False)

# IMDb ratings
We extract the IMDb ratings for movies to know user's opinions on it. 

In [40]:
!curl -o title.ratings.tsv.gz https://datasets.imdbws.com/title.ratings.tsv.gz
!gunzip title.ratings.tsv.gz
!mv title.ratings.tsv data/
!rm title.ratings.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7421k  100 7421k    0     0   9.8M      0 --:--:-- --:--:-- --:--:--  9.8M
rm: title.ratings.tsv.gz: No such file or directory


In [41]:
imdb_df = (pd.read_csv("data/title.ratings.tsv", sep='\t')
.rename(columns={
    'tconst': 'imdb_id',
    'averageRating': 'imdb_rating',
    'numVotes': 'imdb_total_votes'})
)

In [42]:
query = '''
SELECT ?movie ?IMDB_ID
WHERE
{
VALUES ?movieType { wd:Q11424 wd:Q506240 }
?movie wdt:P31 ?movieType.
?movie wdt:P345 ?IMDB_ID.

SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''
query_result = requests.get(WIKI_DATA_SERVICE_URL, params = {'format': 'json', 'query': query})
wikidata_imdb_df =pd.DataFrame(query_result.json()['results']['bindings'])
for column in wikidata_imdb_df.columns:
    wikidata_imdb_df[column] = wikidata_imdb_df[column].apply(lambda x: x['value'] if isinstance(x, dict) and 'value' in x else x)

In [43]:
wikidata_imdb_df = (wikidata_imdb_df
                    .assign(
    movie_wikidata_id = lambda x: x.movie.str.split('/').str[-1],
    imdb_id = lambda x: x.IMDB_ID
)
                    .loc[:, ['movie_wikidata_id', 'imdb_id']]
                    )



# Merging the datasets

In [44]:
book_adaptation_df = (cmu_df
                      .merge(wikidata_imdb_df, on='movie_wikidata_id', how='left')
                      .merge(imdb_df, on='imdb_id', how='left')
                      .merge(tmdb_df, on='imdb_id', how='left')
                      .merge(wikidata_df, on='movie_wikidata_id', how='left')
                      .merge(goodreads_df, on=['join_title', 'join_author'], how='left')
                      .merge(cpi_df, left_on='movie_release', right_on='year', how='left')
                      .assign(
    movie_budget = lambda x: x.movie_budget.astype(float),
    movie_revenue = lambda x: x.movie_revenue.fillna(x.movie_revenue_tmdb).astype(float),
    movie_is_adaptation = lambda x: x.book_wikidata_id.notna()
)
                      .assign(
    movie_budget = lambda df: df.movie_budget * df.inflation_adjustment,
    movie_revenue = lambda df: df.movie_revenue * df.inflation_adjustment
)
                      .drop(columns=['imdb_id', 'movie_revenue_tmdb', 'book_goodreads_id',
                                     'year', 'inflation_adjustment', 'join_title', 'join_author'])
                      )

In [45]:
book_adaptation_df.to_csv("data/book_adaptation.csv", index=False)

# Dealing with missing revenue and budget data
During the data cleaning process, we found out that there are too much missing values for the movies revenues and budget so we decided to extract some more data about it from the wikidata database.

In [46]:
# Count of NaNs in movie_wikidata_id
print(f"Total NaN values in movie_wikidata_id: {book_adaptation_df['movie_wikidata_id'].isna().sum()}")

# Check for unique values and potential duplicates
unique_ids = book_adaptation_df['movie_wikidata_id'].nunique()
print(f"Total unique movie_wikidata_id values: {unique_ids}")
print(f"Total rows in original DataFrame: {len(book_adaptation_df)}")


Total NaN values in movie_wikidata_id: 4288
Total unique movie_wikidata_id values: 76544
Total rows in original DataFrame: 82053


In [47]:
WIKI_DATA_SERVICE_URL = 'https://query.wikidata.org/sparql'

def get_movie_budget_revenue(ids, retries=5, backoff_factor=1):
    movie_ids = " ".join([f"wd:{id}" for id in ids])
    query = (
            'SELECT ?movie ?movieLabel ?budget ?revenue WHERE { '
            'VALUES ?movie { ' + movie_ids + ' } '  # Make sure movie_ids is correctly formatted
                                             '?movie wdt:P31 wd:Q11424. '
                                             'OPTIONAL { ?movie wdt:P2130 ?budget. } '  # Budget property
                                             'OPTIONAL { ?movie wdt:P2142 ?revenue. } '  # Revenue property
                                             'SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } '
                                             '}'
    )

    for attempt in range(retries):
        query_result = requests.get(WIKI_DATA_SERVICE_URL, params={'format': 'json', 'query': query})

        if query_result.status_code == 200:
            try:
                json_data = query_result.json()
                df = pd.DataFrame(json_data['results']['bindings'])
                for column in df.columns:
                    df[column] = df[column].apply(lambda x: x.get('value') if isinstance(x, dict) else x)
                return df
            except ValueError as e:
                print("Error decoding JSON:", e)
                return pd.DataFrame()  # Return an empty DataFrame if JSON error

        elif query_result.status_code == 429:
            # Exponential backoff
            wait_time = backoff_factor * (2 ** attempt)
            time.sleep(wait_time)
        else:
            print(f"Request failed with status code {query_result.status_code}")
            return pd.DataFrame()  # Return empty DataFrame if other errors

    print("Max retries reached.")
    return pd.DataFrame()  # Return an empty DataFrame in case of JSON error

In [48]:
# Filter out NaNs and duplicate IDs for querying
unique_ids = book_adaptation_df['movie_wikidata_id'].dropna().unique()

# Query Wikidata in batches
batch_size = 100
movie_df_list = []
for i in range(0, len(unique_ids), batch_size):
    batch_ids = unique_ids[i:i + batch_size]
    fetched_data = get_movie_budget_revenue(pd.Series(batch_ids))
    if not fetched_data.empty:
        movie_df_list.append(fetched_data)

# Concatenate all fetched data into one DataFrame
if movie_df_list:
    movie_rev_budget = pd.concat(movie_df_list, ignore_index=True)
else:
    movie_rev_budget = pd.DataFrame(columns=['movie_wikidata_id', 'budget', 'revenue'])

# Print count check
print(f"Total rows fetched from Wikidata: {len(movie_rev_budget)}")


Total rows fetched from Wikidata: 65551


In [49]:
movie_rev_budget['movie_wikidata_id']=movie_rev_budget['movie']
movie_rev_budget=movie_rev_budget.drop(columns=['movie'])
movie_rev_budget=movie_rev_budget.drop(columns=['movieLabel'])

In [50]:
movie_rev_budget['movie_wikidata_id'] = movie_rev_budget['movie_wikidata_id'].str.replace('http://www.wikidata.org/entity/', '', regex=False)

In [51]:
# Check for duplicates in movie_rev_budget
duplicates_in_movie_rev_budget = movie_rev_budget[movie_rev_budget.duplicated(subset=['movie_wikidata_id'], keep=False)]
print(f"Number of duplicate movie_wikidata_id entries in movie_rev_budget: {len(duplicates_in_movie_rev_budget)}")
# Drop duplicates, keeping the first occurrence
movie_rev_budget = movie_rev_budget.drop_duplicates(subset='movie_wikidata_id', keep='first')

Number of duplicate movie_wikidata_id entries in movie_rev_budget: 1590


In [52]:
movie_rev_budget.to_csv("data/book_adaptation_expanded.csv", index=False)
